# Name of team Members 

#### Total 3 team members
Tejasv Agarwal (2020MT12475)<br>
Kavya Babu (2020MT12082)<br>
Shakun Vaid (2020MT12457)<br>

Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Let us load the data set

In [ ]:
df= pd.read_csv('./mushrooms.csv')


# Data Analysis On Mushroom Data Set¶


In [ ]:
df.head(5)


Let's see basic data first

In [ ]:
df.describe().T


We can see that all of the features are categorical and we can figure out most frequent categories and their frequecy. Lets now see if there's any null value or not



In [ ]:
df.isna().sum()


There 8124 records in 23 columns. Also, there are no null values



# EDA

In [ ]:
df['class'].hist()

We can figure out that there's no unbalacing in our dataset. What a relief :)

# **Cap Surface**


In [ ]:
import seaborn as sns
figsize=(15,8)

plt.title('cap Surface')
sns.countplot(data = df, x='cap-surface',hue='class', order=df['cap-surface'].value_counts().index,)

In [ ]:
df.groupby(['cap-surface'])['class'].value_counts()

g mushrooms has only poisonous mushrooms and are very less in numbers.


# **Cap Shape**

In [ ]:
plt.title('cap shape')
sns.countplot(data = df, x='cap-shape',hue='class', order=df['cap-shape'].value_counts().index)

In [ ]:
df.groupby('cap-shape')['class'].value_counts()

As we can see from the above data that all the s cap-shape mushrooms are edible while call c mushrooms are poisonous. b shaped mushrooms are mostly edible while k have more posionous. Nothing much can be said about x shaped ones

# **Cap Color**

In [ ]:
plt.title('cap-color effect')
sns.countplot(data = df, x='cap-color',hue='class', order=df['cap-color'].value_counts().index,)

In [ ]:
df.groupby(['cap-color'])['class'].value_counts()

# **Bruises**

In [ ]:
plt.title('bruises effect')
sns.countplot(data = df, x='bruises',hue='class', order=df['bruises'].value_counts().index,)

In [ ]:
df.groupby(['bruises'])['class'].value_counts()

Mushrooms that bruises are mostly edible

# **Odor**

In [ ]:
plt.title('odor effect')
sns.countplot(data = df, x='odor',hue='class', order=df['odor'].value_counts().index,)

In [ ]:
df.groupby(['odor'])['class'].value_counts()

# **Gill Attachment**

In [ ]:
plt.title('gill Attachment')
sns.countplot(data = df, x='gill-attachment',hue='class', order=df['gill-attachment'].value_counts().index,)

In [ ]:
df.groupby(['gill-attachment'])['class'].value_counts()


Same type of EDA can be performed for all other features

# Label encoding

In [ ]:
df.columns

In [ ]:
cols = ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat']
for i in cols:
    df[i] = df[i].astype('category')
    df[i] =  df[i].cat.codes

In [ ]:
inputCols = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat']

In [ ]:
df

# Outlier detection using LOF

We will take neighbours as 400 because there are only 2 classes and total 8124 samples. We can take even more neighbours but lets go with 400 for now

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=400)
df['outlier'] = clf.fit_predict(df[inputCols])

#### Lets check how many are local outliers 

In [ ]:
df[df['outlier'] == -1].shape

Lets remove these outliers to check if we can have improved k-mean algorithm

In [ ]:
df = df[df['outlier'] == 1]

# K-mean Clustering

Lets create a cluster array in advance

In [ ]:
import numpy as np 
ca = np.array(df[inputCols])

Function to calculate eucleadean distance

In [ ]:
def calcEucDistance(t1, t2):
    return (sum((t1 - t2)**2))**0.5

We will now write a function to calculate the centroid after each itteration

In [ ]:
def calc(clusters, ca):
    newCent= []
    cluster_df = pd.concat([pd.DataFrame(ca), pd.DataFrame(clusters,columns=['cluster'])], axis=1)
    for c in set(cluster_df['cluster']):
        thisCluster = cluster_df[cluster_df['cluster']==c][cluster_df.columns[:-1]]
        cm = thisCluster.mean(axis=0)
        newCent.append(cm)
    return newCent

We can write a code to give cluster to each point

In [ ]:
def giveClusters(cent, ca):
    cls = []
    for i in range(ca.shape[0]):
        dist = []
        for c in cent:
            dist.append(calcEucDistance(c, ca[i]))
        cluster = [t for t, v in enumerate(dist) if v==min(dist)]
        cls.append(cluster[0])
    return cls

Next we can find out the average distance between points in a cluster to use them for optimization

In [ ]:
import numpy.matlib as matlib
def calcCentDist(clusters, ca):
    ss = []
    cluster_df = pd.concat([pd.DataFrame(ca),pd.DataFrame(clusters, columns=['cluster'])], axis=1)
    for c in set(cluster_df['cluster']):
        thisCluster = cluster_df[cluster_df['cluster']==c][cluster_df.columns[:-1]]
        cm = thisCluster.mean(axis=0)
        meanNew = matlib.repmat(cm, thisCluster.shape[0],1)
        ss.append(np.sum(np.sum((thisCluster - meanNew)**2)))
    return ss

In [ ]:
k = 2
cluster_vars = []
centroids = [ca[i+2] for i in range(k)]
clusters = giveClusters(centroids, ca)
# initial_clusters = clusters
print(0, round(np.mean(calcCentDist(clusters, ca))))
for i in range(8):
    centroids = calc(clusters, ca)
    clusters = giveClusters(centroids, ca)
    cluster_var = np.mean(calcCentDist(clusters,ca))
    cluster_vars.append(cluster_var)
    print(i+1, round(cluster_var))

In [ ]:
def calFreq(cluster):
    c = {}
    for i in cluster:
        c[i] = c.get(i, 0) + 1
    return c

In [ ]:
calFreq(clusters)

k mean is not performing well as we can see that it is taking huge class imbalacing while creating unsupervised learning model

# Train a classifier

In [ ]:
X = df.drop(['class'], axis = 1)
y = df['class']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth =5, random_state = 42)
clf.fit(X_train, y_train)

In [ ]:
test_pred_decision_tree = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,test_pred_decision_tree)
matrix

Decision tree is performing really well for our use case 